In [1]:
from sdRDM import DataModel
from sdrdm_database import DBConnector, create_tables

In [2]:
# Establish a connection to the database
db = DBConnector(
    username="root",
    password="root",
    host="localhost",
    db_name="db",
    port=3306,
    dbtype="mysql",
)

In [3]:
db.connection.tables

Tables
------
- Test
- nested

In [3]:
# Fetch the sdRDM schema
lib = DataModel.from_markdown("./model.md")
lib.Root.meta_tree()

Root
├── id
├── value
├── nested
│   └── 0
│       ├── id
│       └── another_value
└── some_values


ClassNode(/Root, class_name=None, constants={}, id=None, module=None, outer_type=None)

In [7]:
# Create tables for the sdRDM schema
create_tables(db_connector=db, model=lib.Root)

In [8]:
db.connection.tables

Tables
------
- Root
- nested
- some_values

In [9]:
# We are now seeking a way to insert an instance of 
# the sdRDM schema into the database.
print(db.connection.table("Root"))
print(db.connection.table("nested"))
print(db.connection.table("some_values"))

┏━━━━━━━━━┳━━━━━━━━━┓
┃ value   ┃ id      ┃
┡━━━━━━━━━╇━━━━━━━━━┩
│ float64 │ !string │
└─────────┴─────────┘



┏━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┓
┃ another_value ┃ id      ┃ Root   ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━┩
│ string        │ !string │ string │
└───────────────┴─────────┴────────┘



┏━━━━━━━━━━━━━┳━━━━━━━━┓
┃ some_values ┃ Root   ┃
┡━━━━━━━━━━━━━╇━━━━━━━━┩
│ !string     │ string │
└─────────────┴────────┘



In [10]:
dataset1 = lib.Root(value=100.0, some_values=["Hello", "World"])
dataset1.add_to_nested(another_value="something")
dataset1.add_to_nested(another_value="something else")

print(dataset1)

Root
├── id = root0
├── value = 100.0
├── nested
│   ├── 0
│   │   └── Nested
│   │       ├── id = nested0
│   │       └── another_value = something
│   └── 1
│       └── Nested
│           ├── id = nested1
│           └── another_value = something else
└── some_values = [Hello, World, ...]



In [11]:
dataset2 = lib.Root(value=20.0, some_values=["Whats", "up"])
dataset2.add_to_nested(another_value="hi there")
dataset2.add_to_nested(another_value="this is dataset 2")

print(dataset2)

Root
├── id = root1
├── value = 20.0
├── nested
│   ├── 0
│   │   └── Nested
│   │       ├── id = nested2
│   │       └── another_value = hi there
│   └── 1
│       └── Nested
│           ├── id = nested3
│           └── another_value = this is dataset 2
└── some_values = [Whats, up, ...]



In [12]:
# Insert the new dataset into the database
db.insert(dataset1, dataset2, verbose=True)

Added dataset Root (c100125e-2e40-479c-b1ae-013ca5c25ac5)
Added dataset Root (36b3856c-bf93-43c5-b906-7107100e42c4)


In [13]:
# Check both tables
db.connection.table("Root")

┏━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ value   ┃ id                                   ┃
┡━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ float64 │ !string                              │
├─────────┼──────────────────────────────────────┤
│    20.0 │ 36b3856c-bf93-43c5-b906-7107100e42c4 │
│   100.0 │ c100125e-2e40-479c-b1ae-013ca5c25ac5 │
└─────────┴──────────────────────────────────────┘

In [14]:
db.connection.table("nested")

┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ another_value     ┃ id                                   ┃ Root                                 ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ string            │ !string                              │ string                               │
├───────────────────┼──────────────────────────────────────┼──────────────────────────────────────┤
│ something         │ 095b1d58-8c9a-48f5-8671-53025f027ccd │ c100125e-2e40-479c-b1ae-013ca5c25ac5 │
│ something else    │ 34b48cdf-745f-432f-a39a-29b66fe0fb77 │ c100125e-2e40-479c-b1ae-013ca5c25ac5 │
│ this is dataset 2 │ 438375a0-353b-4192-96ce-6676181e0f0b │ 36b3856c-bf93-43c5-b906-7107100e42c4 │
│ hi there          │ 8bff4b27-2768-4581-a897-f50002570ae1 │ 36b3856c-bf93-43c5-b906-7107100e42c4 │
└───────────────────┴──────────────────────────────────────┴──────────────────────────────────────┘

In [15]:
db.connection.table("some_values")

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ some_values ┃ Root                                 ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ !string     │ string                               │
├─────────────┼──────────────────────────────────────┤
│ Hello       │ c100125e-2e40-479c-b1ae-013ca5c25ac5 │
│ World       │ c100125e-2e40-479c-b1ae-013ca5c25ac5 │
│ Whats       │ 36b3856c-bf93-43c5-b906-7107100e42c4 │
│ up          │ 36b3856c-bf93-43c5-b906-7107100e42c4 │
└─────────────┴──────────────────────────────────────┘